<a href="https://colab.research.google.com/github/meghana-anupoju/AI-Study-Buddy/blob/main/AI_Study_Buddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-adk google-genai


In [33]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY") # Load from Colab secrets
MODEL_NAME = "gemini-2.5-flash-lite"

In [34]:
from google.genai import types
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.tools import google_search, AgentTool
from google.adk.runners import Runner, InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.genai import types as genai_types


In [35]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=2,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)


In [36]:
concept_tutor = Agent(
    name="concept_tutor",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    description="Explains programming and CS concepts step-by-step.",
    instruction="""
    You are a friendly concept tutor. Always:
    1. Ask for the student’s level if unclear.
    2. Break the concept into steps.
    3. Use analogies and examples.
    4. End with a quick recap.
    """,
)


In [37]:
quiz_agent = Agent(
    name="quiz_agent",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    description="Creates quizzes and evaluates answers.",
    instruction="""
    You generate quizzes (MCQs or short answers).
    When checking answers:
    - Provide score
    - Detailed feedback
    - What to revise
    """,
)


In [38]:
coach_agent = Agent(
    name="coach_agent",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    description="Gives motivation and next study suggestions.",
    instruction="""
    You read the session context and act like a study coach.
    Give next steps, resources and encouragement.
    Keep it short and practical.
    """,
)


In [39]:
study_notes = {}

def save_note(topic: str, summary: str):
    """
    Saves 3–4 line summary notes for the student.
    """
    study_notes[topic.lower()] = summary
    return {"status": "saved", "topic": topic, "summary": summary}


In [40]:
root_agent = Agent(
    name="ai_study_buddy",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    description="Main AI tutor that delegates to sub-agents.",
    instruction="""
    You are the AI Study Buddy. You decide which agent to call:

    - concept_tutor → if user says: explain, teach, understand
    - quiz_agent → if user says: quiz me, test me, check my answers
    - coach_agent → if user says: guide me, what next, I’m confused

    If unclear, ask a clarifying question.
    Always be friendly and call the correct tool.
    """,
    tools=[
        AgentTool(agent=concept_tutor),
        AgentTool(agent=quiz_agent),
        AgentTool(agent=coach_agent),
        save_note
    ]
)


In [41]:
session_service = InMemorySessionService()

runner = Runner(
    agent=root_agent,
    app_name="ai_tutor_app",
    session_service=session_service
)

USER_ID = "student_001"
SESSION_ID = "session_001"


In [42]:
async def chat(message: str):
    print(f"\nYou: {message}")

    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=genai_types.Content(
            role="user", parts=[genai_types.Part(text=message)]
        )
    ):
        if event.content and event.content.parts:
            print("Tutor:", event.content.parts[0].text)


In [43]:
await session_service.create_session(app_name=runner.app_name, user_id=USER_ID, session_id=SESSION_ID)

await chat("Hi! I'm a 2nd year student struggling with time complexity.")
await chat("Explain Big O Notation like I'm a beginner.")
await chat("Quiz me on this.")
await chat("My answers: 1) B, 2) C, 3) A. Check them.")
await chat("What should I study next?")


You: Hi! I'm a 2nd year student struggling with time complexity.
Tutor: Hi there! I can help with that. The `concept_tutor` agent can explain concepts like time complexity. Would you like me to ask it to explain time complexity to you? Or perhaps you'd like to try a quiz on it with `quiz_agent`? Or if you're feeling stuck, `coach_agent` could offer some guidance.

You: Explain Big O Notation like I'm a beginner.
Tutor: None
Tutor: None

You: Quiz me on this.
Tutor: Sure thing! What would you like to be quizzed on?

You: My answers: 1) B, 2) C, 3) A. Check them.
Tutor: What was the test on?

You: What should I study next?
Tutor: None
Tutor: None


In [ ]:
await chat("Hi! I'm a 2nd year student struggling with time complexity.")
await chat("Explain Big O Notation like I'm a beginner.")
await chat("Quiz me on this.")
await chat("My answers: 1) B, 2) C, 3) A. Check them.")
await chat("What should I study next?")



You: Hi! I'm a 2nd year student struggling with time complexity.
Tutor: None
Tutor: None

You: Explain Big O Notation like I'm a beginner.
Tutor: None
Tutor: None

You: Quiz me on this.
Tutor: Sure thing! What topic would you like to be quizzed on?

You: My answers: 1) B, 2) C, 3) A. Check them.
Tutor: Please share the questions first, and I'll be happy to check your answers for you!

You: What should I study next?
Tutor: To help me suggest what you should study next, could you tell me what programming language or concepts you're currently working with or have recently studied?


In [44]:
session = await session_service.get_session(
    app_name="ai_tutor_app",
    user_id=USER_ID,
    session_id=SESSION_ID
)

for e in session.events:
    print(e)


model_version=None content=Content(
  parts=[
    Part(
      text="Hi! I'm a 2nd year student struggling with time complexity."
    ),
  ],
  role='user'
) grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='e-93410941-75d1-4275-967d-1b5dbcc4ebd9' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=None, end_of_agent=None, agent_state=None, rewind_before_invocation_id=None) long_running_tool_ids=None branch=None id='c46a828a-a8c8-4193-a591-49a0ac8650e0' timestamp=1764403122.915571
model_version='gemini-2.5-flash-lite' content=Content(
  par

In [45]:
study_notes


{}